In [1]:
import pandas as pd
from numpy import inf, NaN, where
from datetime import datetime
from os import listdir, path
from pycovid import pycovidfunc as cv

In [2]:
pd.options.display.max_columns = None
pd.options.display.max_rows = None

# 1 - DATA TO BE READ BY TABLEAU

In [2]:
config=pd.read_csv('config.csv',index_col='var')
file = config.loc['raw_data'].path

In [3]:
df = pd.read_csv(file)
df = df[df['Province/State'] != '-']

In [4]:
columns = ['Province/State','Country/Region','Date','Confirmed','Active','Recovered','Deaths']
df = df[columns].groupby(['Province/State','Country/Region','Date']).sum().reset_index()

In [5]:
columns = ['Confirmed','Active','Recovered','Deaths']
new_cases = [item + ' new cases' for item in columns]

columns_mov_avg = columns.copy()
columns_mov_avg.extend(new_cases)

mov_avg = 3
columns_mov_avg = [item + ' {}-day mov avg'.format(mov_avg) for item in columns_mov_avg]

df[new_cases] = df.groupby('Province/State')[columns].diff().fillna(value=0)
df_aux = df.groupby('Province/State').rolling(3).mean().fillna(value=0).reset_index()
df_aux.drop(['Province/State','level_1'],axis=1,inplace=True)

df_aux.columns = columns_mov_avg
df = pd.concat([df,df_aux],axis=1)

In [22]:
df.to_excel('province_report.xlsx')

In [17]:
df[df['Province/State'] == 'Quebec'].head(60)

,Province/State,Country/Region,Date,Confirmed,Active,Recovered,Deaths,Confirmed new cases,Active new cases,Recovered new cases,Deaths new cases,Confirmed 3-day mov avg,Active 3-day mov avg,Recovered 3-day mov avg,Deaths 3-day mov avg,Confirmed new cases 3-day mov avg,Active new cases 3-day mov avg,Recovered new cases 3-day mov avg,Deaths new cases 3-day mov avg
44372,Quebec,Canada,2020-03-09,3.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000
44373,Quebec,Canada,2020-03-10,4.0,4.0,0.0,0.0,1.0,1.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000
44374,Quebec,Canada,2020-03-11,8.0,8.0,0.0,0.0,4.0,4.0,0.0,0.0,5.000000,5.000000,0.0,0.000000,1.666667,1.666667,0.0,0.000000
44375,Quebec,Canada,2020-03-12,9.0,9.0,0.0,0.0,1.0,1.0,0.0,0.0,7.000000,7.000000,0.0,0.000000,2.000000,2.000000,0.0,0.000000
44376,Quebec,Canada,2020-03-13,17.0,17.0,0.0,0.0,8.0,8.0,0.0,0.0,11.333333,11.333333,0.0,0.000000,4.333333,4.333333,0.0,0.000000
44377,Quebec,Canada,2020-03-14,17.0,17.0,0.0,0.0,0.0,0.0,0.0,0.0,14.333333,14.333333,0.0,0.000000,3.000000,3.000000,0.0,0.000000
44378,Quebec,Canada,2020-03-15,24.0,24.0,0.0,0.0,7.0,7.0,0.0,0.0,19.333333,19.333333,0.0,0.000000,5.000000,5.000000,0.0,0.000000
44379,Quebec,Canada,2020-03-16,50.0,50.0,0.0,0.0,26.0,26.0,0.0,0.0,30.333333,30.333333,0.0,0.000000,11.000000,11.000000,0.0,0.000000
44380,Quebec,Canada,2020-03-17,74.0,74.0,0.0,0.0,24.0,24.0,0.0,0.0,49.333333,49.333333,0.0,0.000000,19.000000,19.000000,0.0,0.000000
44381,Quebec,Canada,2020-03-18,94.0,94.0,0.0,0.0,20.0,20.0,0.0,0.0,72.666667,72.666667,0.0,0.000000,23.333333,23.333333,0.0,0.000000


In [7]:
df.head(60)

,Province/State,Country/Region,Date,Confirmed,Active,Recovered,Deaths,Confirmed new cases,Active new cases,Recovered new cases,Deaths new cases,Confirmed 3-day mov avg,Active 3-day mov avg,Recovered 3-day mov avg,Deaths 3-day mov avg,Confirmed new cases 3-day mov avg,Active new cases 3-day mov avg,Recovered new cases 3-day mov avg,Deaths new cases 3-day mov avg
0,"Montreal, QC",Canada,2020-02-28,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,"Montreal, QC",Canada,2020-02-29,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,"Montreal, QC",Canada,2020-03-01,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,"Montreal, QC",Canada,2020-03-02,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,"Montreal, QC",Canada,2020-03-03,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,"Montreal, QC",Canada,2020-03-04,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,"Montreal, QC",Canada,2020-03-05,2.0,2.0,0.0,0.0,1.0,1.0,0.0,0.0,1.333333,1.333333,0.000000,0.000000,0.333333,0.333333,0.000000,0.000000
7,"Montreal, QC",Canada,2020-03-06,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,1.666667,1.666667,0.000000,0.000000,0.333333,0.333333,0.000000,0.000000
8,"Montreal, QC",Canada,2020-03-07,3.0,3.0,0.0,0.0,1.0,1.0,0.0,0.0,2.333333,2.333333,0.000000,0.000000,0.666667,0.666667,0.000000,0.000000
9,"Montreal, QC",Canada,2020-03-08,4.0,4.0,0.0,0.0,1.0,1.0,0.0,0.0,3.000000,3.000000,0.000000,0.000000,0.666667,0.666667,0.000000,0.000000


In [14]:
df['Province/State'].loc[df['Country/Region'] == 'Sweden'].unique()

array(['Blekinge', 'Dalarna', 'Gavleborg', 'Gotland', 'Halland',
       'Jamtland Harjedalen', 'Jonkoping', 'Kalmar', 'Kronoberg',
       'Norrbotten', 'Orebro', 'Ostergotland', 'Skane', 'Sormland',
       'Stockholm', 'Uppsala', 'Varmland', 'Vasterbotten',
       'Vasternorrland', 'Vastmanland', 'Vastra Gotaland'], dtype=object)

In [5]:
def world_data_formatter(raw_data):
    '''
    Creates the world data report from the raw data dataframe.
    
    This function works along the raw_data as returned by the
    raw_data_formatter function. Changes in raw_data_formatter
    affect directly this function.
    
    It creates all columns necessary for analysis with Tableau
    from the John Hopkins Data Science Center and it returns a
    new DataFrame object with calculated columns.
    
    Parameters
    ----------
    raw_data: obj, DataFrame
        the raw data DataFrame as returned by the raw_data_formatter
        function.
    '''
    import pandas as pd
    from numpy import inf, NaN, where
    from datetime import datetime
    
    df_by_country = df.groupby(['Country/Region','Date']).sum().reset_index()

    # Calculating the number of days since the 1st case:
    df_by_country['Days_since_1st_case'] = df_by_country['Date']

    countries = df_by_country['Country/Region'].unique()
    for country in countries:
        idx = where(df_by_country['Country/Region'] == country)
        first_date = pd.to_datetime(df_by_country['Date'].loc[min(idx[0])])
        for index in idx[0]:
            date_diff = (pd.to_datetime(df_by_country.at[index,'Days_since_1st_case']) - 
                         first_date).days
            df_by_country.at[index,'Days_since_1st_case'] = date_diff


    # columns over which the calculations will be performed
    root_columns = ['Active','Confirmed','Deaths','Recovered']

    # creating columns of daily percentage of increase in values:
    for col in root_columns:
        col_daily_inc = col + "_daily_%inc_by_country"
        col_new_cases = col + '_new_cases'
        col_new_cases_inc = col + '_new_cases_inc_rate'
        col_new_cases_inc_speed = col + '_new_cases_inc_rate_speed'

        df_by_country[col_new_cases] = (df_by_country[col] - 
                                        df_by_country[col].shift(periods=1)
                                       ).fillna(value=0)
        df_by_country[col_daily_inc] = df_by_country[col].pct_change().replace([inf, NaN], 0)*100

        # 1st derivative of column datas. It represents the rate of change in new cases:
        df_by_country[col_new_cases_inc] = (df_by_country[col_new_cases] - 
                                            df_by_country[col_new_cases].shift(periods=1)
                                           ).fillna(value=0)        

        # 2nd derivative of column datas. It represents the acceleration of the increase rate
        # of the new cases:
        df_by_country[col_new_cases_inc_speed] = (df_by_country[col_new_cases_inc] -
                                                  df_by_country[col_new_cases_inc].shift(periods=1)
                                                  ).fillna(value=0)
        
    return df_by_country

In [6]:
df_by_country.to_json(config.loc['formatted_data'].path,orient='records')

# DATA FOR PYTHON USE

In [5]:
# drop columns that won't be used in the calculations:
column_labels = ['Last Update','Latitude','Longitude','Year','Month','Week','Day','Mortality rate in %']
df_formatted.drop(axis=1,inplace=True,labels=column_labels)

df_by_country = df_formatted.groupby(['Country/Region','Date']).sum().reset_index()

# Formatting dataframe for further calculations:
root_columns = ['Active','Confirmed','Deaths','Recovered']       # columns over which the calculations will be performed
MA = [3,7,15]                                                    # Moving Average intervals for computation

# creating columns of daily percentage of increase in values:
for col in root_columns:
    df_aux = df_by_country.groupby('Country/Region')

    country_daily_inc = col + "_daily_%inc_by_country"
   
    df_by_country[country_daily_inc] = df_aux[col].pct_change().fillna(value=0)*100
    for interval in MA:
        interval_col_name = col + "_" + str(interval) + "day_MA"
        df_by_country[interval_col_name] = df_aux.rolling(interval)[col].mean().fillna(method='bfill').reset_index(drop=True)    
        
        if interval == min(MA):
            column_name_first = col + '_1st_derivative'
            column_name_second = col + '_2nd_derivative'

            df_by_country[column_name_first] = (df_by_country[interval_col_name] - df_by_country[interval_col_name].shift(periods=1)).fillna(method='bfill')
            df_by_country[column_name_second] = (df_by_country[column_name_first] - df_by_country[column_name_first].shift(periods=1)).fillna(method='bfill')        

In [6]:
df_by_country.sort_values(by=['Country/Region','Date']).head(5)

,Country/Region,Date,Active,Confirmed,Deaths,Recovered,Active_daily_%inc_by_country,Active_3day_MA,Active_1st_derivative,Active_2nd_derivative,Active_7day_MA,Active_15day_MA,Confirmed_daily_%inc_by_country,Confirmed_3day_MA,Confirmed_1st_derivative,Confirmed_2nd_derivative,Confirmed_7day_MA,Confirmed_15day_MA,Deaths_daily_%inc_by_country,Deaths_3day_MA,Deaths_1st_derivative,Deaths_2nd_derivative,Deaths_7day_MA,Deaths_15day_MA,Recovered_daily_%inc_by_country,Recovered_3day_MA,Recovered_1st_derivative,Recovered_2nd_derivative,Recovered_7day_MA,Recovered_15day_MA
0,Azerbaijan,2020-02-28 00:00:00,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.4,0.0,1.0,0.0,0.0,1.0,1.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Afghanistan,2020-02-24 00:00:00,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.4,0.0,1.0,0.0,0.0,1.0,1.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Afghanistan,2020-02-25 00:00:00,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.4,0.0,1.0,0.0,0.0,1.0,1.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Afghanistan,2020-02-26 00:00:00,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.4,0.0,1.0,0.0,0.0,1.0,1.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Afghanistan,2020-02-27 00:00:00,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.4,0.0,1.0,0.0,0.0,1.0,1.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
df_by_country.to_json('country_report.json',orient='records')

In [1]:
import pandas as pd

In [2]:
file_dir = r'C:\Users\user\Documents\GitHub\COVID-19\consolidated_data/country_report.json'

In [3]:
df = pd.read_json(file_dir)

In [4]:
df.head(10)

,Country/Region,Date,Active,Confirmed,Deaths,Latitude,Longitude,Recovered,Mortality rate in %,Days_since_1st_case,...,Confirmed_new_cases_inc_rate,Confirmed_new_cases_inc_rate_speed,Deaths_new_cases,Deaths_daily_%inc_by_country,Deaths_new_cases_inc_rate,Deaths_new_cases_inc_rate_speed,Recovered_new_cases,Recovered_daily_%inc_by_country,Recovered_new_cases_inc_rate,Recovered_new_cases_inc_rate_speed
0,Afghanistan,2020-02-24,1,1,0,0.0,0.0,0,0.0,0,...,0,0,0,0.0,0,0,0,0.0,0,0
1,Afghanistan,2020-02-25,1,1,0,0.0,0.0,0,0.0,1,...,0,0,0,0.0,0,0,0,0.0,0,0
2,Afghanistan,2020-02-26,1,1,0,0.0,0.0,0,0.0,2,...,0,0,0,0.0,0,0,0,0.0,0,0
3,Afghanistan,2020-02-27,1,1,0,0.0,0.0,0,0.0,3,...,0,0,0,0.0,0,0,0,0.0,0,0
4,Afghanistan,2020-02-28,1,1,0,0.0,0.0,0,0.0,4,...,0,0,0,0.0,0,0,0,0.0,0,0
5,Afghanistan,2020-02-29,1,1,0,0.0,0.0,0,0.0,5,...,0,0,0,0.0,0,0,0,0.0,0,0
6,Afghanistan,2020-03-01,1,1,0,33.0,65.0,0,0.0,6,...,0,0,0,0.0,0,0,0,0.0,0,0
7,Afghanistan,2020-03-02,1,1,0,33.0,65.0,0,0.0,7,...,0,0,0,0.0,0,0,0,0.0,0,0
8,Afghanistan,2020-03-03,1,1,0,33.0,65.0,0,0.0,8,...,0,0,0,0.0,0,0,0,0.0,0,0
9,Afghanistan,2020-03-04,1,1,0,33.0,65.0,0,0.0,9,...,0,0,0,0.0,0,0,0,0.0,0,0


In [5]:
save_dir = r'C:\Users\user\Documents\GitHub\COVID-19\SQL/country_report.csv'

In [6]:
df.to_csv(save_dir,index=False)

In [14]:
file_name = r'C:\Users\user\Documents\GitHub\COVID-19\csse_covid_19_data\csse_covid_19_daily_reports\08-17-2020.csv'
country_report = r'C:\Users\user\Documents\GitHub\COVID-19\consolidated_data\country_report.json'

In [16]:
df=pd.read_csv(file_name)
df2=pd.read_json(country_report)

In [6]:
df_aux = df.copy()

In [7]:
df_aux = df_aux[['Province_State','Country_Region']]

In [17]:
df_aux.loc[df_aux['Country_Region'] == 'China']

,Province_State,Country_Region
98,Anhui,China
99,Beijing,China
100,Chongqing,China
101,Fujian,China
102,Gansu,China
103,Guangdong,China
104,Guangxi,China
105,Guizhou,China
106,Hainan,China
107,Hebei,China


In [18]:
df2['Country/Region'].unique().size

200